In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PowerTransformer

In [2]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')
testOriginal = pd.read_csv('../test.csv')

In [3]:
train.shape

(181507, 272)

In [4]:
test.drop('row ID', axis=1, inplace=True)

<h3>SubArea Removal<h3>

In [5]:
test.drop('sub_area', axis=1, inplace=True)
train.drop('sub_area', axis=1, inplace=True)

<h1>Categorical To Numerical<h1>

<h3>Label<h3>

In [6]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# categorical_columns = train.select_dtypes(include=['object']).columns.tolist()
# print("Train: Categorical columns:", categorical_columns)

# label_encoder = LabelEncoder()

# for column in categorical_columns:
#     train[column] = label_encoder.fit_transform(train[column])

# categorical_columns_test = test.select_dtypes(include=['object']).columns.tolist()
# print("Test: Categorical columns:", categorical_columns_test)

# label_encoder = LabelEncoder()

# for column in categorical_columns_test:
#     test[column] = label_encoder.fit_transform(test[column])

<h1>Working<h1>

In [7]:
X = train.loc[:, train.columns != 'price_doc']
y = train[['price_doc']]

# scaler = RobustScaler()
# X = scaler.fit_transform(X)
# test = scaler.transform(test)  

In [8]:
X.shape

(181507, 287)

In [9]:
test.shape

(77789, 287)

<h1>Forward Feature Selection<h1>

In [10]:
X2 = X[['full_sq', 'floor', 'build_count_monolith', 'industrial_km', 'trc_sqm_500',
 'mosque_count_500', 'leisure_count_500', 'office_sqm_1000',
 'cafe_count_1000_price_high', 'leisure_count_1000', 'power_transmission_line_km', 
 'big_market_km', 'public_healthcare_km', 'workplaces_km']]
test2 = test[['full_sq', 'floor', 'build_count_monolith', 'industrial_km', 'trc_sqm_500',
 'mosque_count_500', 'leisure_count_500', 'office_sqm_1000',
 'cafe_count_1000_price_high', 'leisure_count_1000', 'power_transmission_line_km', 
 'big_market_km', 'public_healthcare_km', 'workplaces_km']]

<h1>Min_Max Scaling<h1>

In [11]:
scaler = StandardScaler()
X3 = scaler.fit_transform(X2)
test3 = scaler.transform(test2)  

pca = PCA(n_components=10)
principalComponents = pca.fit_transform(X3)
X3 = pd.DataFrame(data = principalComponents)

pca2 = PCA(n_components=10)
principalComponents = pca2.fit_transform(test3)
test3 = pd.DataFrame(data = principalComponents)


poly = PolynomialFeatures(2)
X3 = poly.fit_transform(X3)
test3 = poly.fit_transform(test3)


<h1>Applying Model<h1>

In [12]:
X3.shape

(181507, 66)

In [13]:
test3.shape

(77789, 66)

In [14]:



# Create and train the Lasso regression model
alpha_value = 0.5  # You can adjust the alpha parameter based on your needs
lasso_reg = Lasso(alpha=alpha_value, random_state=42)
lasso_reg.fit(X3, y)

# Print the coefficients and intercept
print("Coefficients:", lasso_reg.coef_)
print("Intercept:", lasso_reg.intercept_)

# Make predictions on the test data
y_pred_lasso = lasso_reg.predict(test3)

Coefficients: [ 0.00000000e+00  8.90504004e+06 -4.82768126e+06 -7.08340240e+05
 -5.75894596e+05  2.03369789e+06 -1.67996826e+06 -3.58097000e+06
 -5.08618418e+05 -1.35621738e+06 -4.58079891e+05 -5.01287555e+05
  6.63278246e+05  7.39515969e+04  9.33559150e+04 -3.59965276e+05
  2.75288134e+05  5.27889890e+05  8.32931879e+04  1.65041090e+05
  5.55150952e+04 -2.90676485e+05 -1.22242317e+05 -1.72464319e+05
 -3.52363046e+05  4.62990131e+05 -1.61084157e+05 -2.32519458e+04
 -5.20045573e+03  1.09182817e+05 -8.85266249e+04  9.84355468e+04
  1.56233603e+05 -2.60812298e+05 -6.48198681e+04  8.14591611e+04
 -1.21333842e+05 -1.09311490e+03  2.37476077e+05 -1.08579722e+05
  1.07382615e+05  2.24172452e+05  1.36399729e+05 -1.24273639e+05
  7.68239661e+04  1.02689779e+05  5.01493743e+04  3.89027112e+05
  1.91309211e+05  6.44881053e+04  1.24153772e+05  2.76945915e+05
 -2.45299342e+04  2.18166986e+05  9.37098745e+03  2.17149250e+04
 -5.04991363e+05 -6.99112567e+05 -5.82423392e+05  2.26555079e+05
 -1.0968005

In [15]:
# Save the predictions to a CSV file
result_df_lasso = pd.DataFrame({'row ID': testOriginal['row ID'], 'price_doc': y_pred_lasso.flatten()})
result_df_lasso.to_csv('Day_6_Abdullah_Maqsood_24448_Submission_10.csv', index=False)